# Create search strings

In [3]:
%run lit_utility_functions_2025.ipynb

import bream
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words
from pyinflect import getInflection
import spacy
from textblob import Word


# Load NLTK resources
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
try:
    nltk.data.find('corpora/words')
except LookupError:
    nltk.download('words')
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    
# Load spaCy model
try:
    spacy_nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading en_core_web_sm model...")
    spacy.cli.download("en_core_web_sm")
    spacy_nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\messa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
def find_matching_words(pattern, bound_pattern=True):
    """
      Finds all English words that match a given regular expression pattern.
      Args:
        pattern: The regular expression pattern (string).
      Returns:
        A list of English words that match the pattern.  Returns an empty list if no
        words match or if there's an invalid regex pattern.  Prints a warning if the
        NLTK words corpus is not found.
    """
    if bound_pattern:
        pattern = f'^{pattern}$'
    regex = re.compile(pattern)
    english_words = words.words()
    matching_words = [word for word in english_words if regex.search(word)]

    return matching_words


def combinate_concats(
    prefixes, suffixes, separators=[" ", "-", ""], add_quotes=False):
    """
    Generates all combinations using itertools.product (most efficient).
    """
    if add_quotes:
        combinations = [
            '"' + "".join(combination) + '"'
            for combination in itertools.product(prefixes, separators, suffixes)
        ]
    else:
        combinations = [
            "".join(combination)
            for combination in itertools.product(prefixes, separators, suffixes)
        ]
    return combinations

def plural_form_exists(word, in_lemmatizer):
    """
    Checks if a plausible plural form of a word exists, with improved logic
    and handling of irregular plurals.  Uses WordNet and a rule-based fallback.

    Args:
      word: The word (string) to check.

    Returns:
      True if a plausible plural form is found, False otherwise.
    """
    # 1. Check if the word is already plural (common case):
    if wordnet.synsets(word) and any(lemma.name().endswith('s') 
                                     for synset in wordnet.synsets(word)
                                     for lemma in synset.lemmas()):
       return True

    # 2. Lemmatize the word (get the singular form):
    lemma = in_lemmatizer.lemmatize(word, wordnet.NOUN)


    #3. Check if word is the lemma (if so, it is most likely singular)
    if lemma != word:
        return True #Word is not the lemma (it's likely already a plural form)
    
    # 4. If lemma and word are the same, then add an s and try again with wordnet
    if wordnet.synsets(word + 's'):
        return True
    
    # 5. Try common plural endings
    if word.endswith(("s", "x", "z", "ch", "sh")):
        plural = word + "es"
    elif word.endswith("y") and len(word) > 1 and word[-2] not in "aeiou":
        plural = word[:-1] + "ies"
    else:
        plural = word + "s"
    
    if wordnet.synsets(plural):
        return True

    return False
    
def textblob_pluralize(word):
    w = Word(word)
    return w.pluralize()

def get_spelling_variants(word):
    """Gets American and British spelling variants of a word using bream."""
    variants = set()
    variants.add(word)  # Add the original word

    try:
      #The following lines will generate errors if the words are not in the
        #dictionary. We capture these.
        american = bream.to_american(word)
        variants.add(american)
    except:
        pass
    try:
        british = bream.to_british(word)
        variants.add(british)
    except:
      pass
    return list(variants)

In [7]:
def create_generic_search_terms():
    combo1_1 = ["ecologic\\S*", ["eco", "hydrologic\\S*"], 
                ["hydro", "ecologic\\S*"],
                'environmental', 'minim\\S\\S', 'acceptable',
                'augmented', 'augmentation', 'compensation', 
                'experimental', 'flushing', ['in', 'stream'], 'maintenance',
                'optimum', 'restorati\\S{2}']

    combo1_2 = ['flood', 'flow', ['water', 'level'], 'discharge']
    
    combo2_1 = ['compensat[a-z]{1,3}', 'conservation', 'cultural', ['cut', 'off'], 
                'design', 'fish', 'functional', 'indigenous', 'limit', 'maintenance',
                'management', 'maximum', 'natural', 'preference', 
                'protection', 'rating', 'regime[a-z]{0,1}', 'residual',
                'right', 'sanita(ry|tion)', 'scenario', 'standard', 
                'suitable', 'surplus', 'sustainable', 'threshold',
                'use', 'vital']
    combo2_2 = ['flow']
    
    combo3_1 = ['downstream', 'dam', 'reservoir']
    combo3_2 = [['water', 'release'], ['flow', 'release'], 'reoperation']
    
    combo4_1 = ['controlled', 'artificial']
    combo4_2 = ['flood']
    
    combo5_1 = ['hydrologic(al)*']
    combo5_2 = ['requirement', 'manipulation']
    
    combo6_1 = ['flow', ['stream', 'flow'], 'freshwater', 'water', ['water', 'level']]
    combo6_2 = ['abstraction', 'allocation', 'criteri\\S{1,2}', 'delivery*', 
                'demand', 'guideline',
                'need', 'prescription', 'recommendation', 'recovery', 'requirement', 
                'reserve', 'restoration', 'restriction', 'withdrawal']
    
    search_dict = {
         'search1': ['with', [combo1_1, combo1_2]],
         'search2':  ['with', [combo2_1, combo2_2]],
         'search3': ['pre', [combo3_1, combo3_2]],
         'search4':  ['pre', [combo4_1, combo4_2]],
         'search5':  ['pre', [combo5_1, combo5_2]],
         'search6':  ['with', [combo6_1, combo6_2]]
    }
    return(search_dict)

In [9]:
def create_search_string(in_search_duo, inflect, 
                         or_chars = ' OR ', and_chars = ' AND ',
                         inner_separators = [" ", "-", ""], use_quotes = True
                        ):
    search_list_formatted = []

    if inflect:
        nltk_lemmatizer = WordNetLemmatizer()
  
    #For each combo item~~~~~~~~~~~~~~~~~~~~
    combo_list_formatted = []
    for combo_list in in_search_duo[1]:
        word_group_formatted = []
        for repattern_group in combo_list:
            #When multiple words in a group
            if isinstance(repattern_group, list):
                #print(repattern_group)
                #Apply find_matching_words to each
                repattern_group_inflected = []
                for repattern in repattern_group:
                    k = find_matching_words(repattern)
                    if (len(k) == 0):
                        k = repattern
                    if not isinstance(k, list):
                        k = [k]

                    if not inflect:
                        #Lemmatize with spacy
                        token_list = spacy_nlp(" ".join(k))
                        k = [token.lemma_ 
                             for token in token_list]

                    repattern_group_inflected.append(k)

                #Then join them in the order with space, no space, and hyphen
                if (in_search_duo[0] == 'with') and use_quotes:
                    use_quotes = True
                else:
                    use_quotes = False
                repattern_group_inflected = combinate_concats(
                    prefixes=repattern_group_inflected[0],
                    suffixes=repattern_group_inflected[1],
                    separators=inner_separators,
                    add_quotes=use_quotes
                )
            else:
                repattern_group_inflected = find_matching_words(repattern_group)
                if (len(repattern_group_inflected) == 0):
                    repattern_group_inflected = repattern_group
                if not isinstance(repattern_group_inflected, list):
                    repattern_group_inflected = [repattern_group_inflected]
                if not inflect:
                    #Lemmatize with spacy
                    token_list = spacy_nlp(" ".join(repattern_group_inflected))
                    repattern_group_inflected = [token.lemma_ 
                                                   for token in token_list]

            #Remove duplicates
            word_group_formatted += list(set(repattern_group_inflected))
            
        # Create a new list to store all variations
        new_word_group = []
        for word in word_group_formatted:
            #if inflect, add all original matching words, plural forms, 
            #present participle, and alternative spelling
            if inflect:
                new_word_group.append(word)  # Add original word
                
                # Add plural form
                if plural_form_exists(word, nltk_lemmatizer):
                    plural_word = textblob_pluralize(word)
                    new_word_group.append(plural_word)

                # Add present participle
                pre_participle = getInflection(word, 'VBG')
                if pre_participle is not None:  # Avoid adding if it's the same
                    if isinstance(pre_participle, list):
                        for w in pre_particile:
                            new_word_group.append(pre_participle)
                    else:
                        new_word_group.append(pre_participle[0])

                # Add spelling variants (british vs americna or vice-versa)
                spelling_variants = get_spelling_variants(word)
                for variant in spelling_variants:
                    if variant != word: # Avoid adding if its the same
                      new_word_group.append(variant)
            else:
              new_word_group.append(word)  # Keep original word

        # Remove duplicates (again, after adding variants)
        word_group_formatted = list(set(new_word_group))

        #Combine word within word_group/combo_list
        combo_list_formatted.append(word_group_formatted)
        
    #Then create actual combinations for each combo duo
    if in_search_duo[0] == 'pre':
        #Create combinations of all terms separated by a space
        search_duo_formatted = combinate_concats(
            combo_list_formatted[0], 
            combo_list_formatted[1],
            separators=" ",
            add_quotes = use_quotes
        )
        search_duo_formatted = recomb(search_duo_formatted, 
                               recomb_sep = or_chars)
        
    elif in_search_duo[0] == 'with':
        #Create two blocks, OR within each block, AND between the two
        search_duo_formatted = [
            f'{recomb(combo_list_formatted[0], recomb_sep=or_chars)}'
            f'{and_chars}'
            f'{recomb(combo_list_formatted[1], recomb_sep=or_chars)}'
        ][0]
    return(search_duo_formatted)